In [2]:
# https://www.geeksforgeeks.org/creating-tabbed-widget-with-python-tkinter/#
# https://www.pythontutorial.net/tkinter/tkinter-treeview/

import tkinter as tk
from tkinter import ttk
import pandas as pd
import csv

# Features we are hoping to implement
#   * Get a random review by clicking a button
#   * Sort tables by clicking on columns
#   * Add data for amenities, etc...
#   * Make reviews scrollable
#   * Improve GUI aesthetics?
#   * Add tab for making quantitative comparisons between locations
#       * Comparisons between average apartment price, reviews, commute, etc?..
#       * May be more practical to process data in a csv, then import as table

class MainWindow:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Binghamton Apartment Comparison Tool")
        self.init_tabs()
        
        self.import_average_ratings()
        self.load_info()
        self.import_info()
        
    def init_tabs(self):
        self.tabs_widget = ttk.Notebook(self.root)
        self.all_tabs = []
        
        # Note that a lot of functions here depends on the order given by this tuple
        # Since it's difficult to match tabs to their name, assume self.all_tabs is a list
        # in the order of [UClubTab, HawleyTab, TwinRiverTab, PrintingHouseTab]
        locations = ("UClub", "20 Hawley", "Twin River Commons", "Printing House")
        for location in locations:
            new_tab = ttk.Frame(self.tabs_widget)
            self.tabs_widget.add(new_tab, text=location)
            self.all_tabs.append(new_tab)
        self.tabs_widget.pack()
    
    def import_average_ratings(self):
        self.average_rating_labels = []
        with open("Reviews/google_average_ratings.csv", newline="") as avg_ratings_file:
            csv_reader = list(csv.reader(avg_ratings_file))
            average_ratings = csv_reader[1]
            for index, rating in enumerate(average_ratings):
                label_text = f"Location Average Online Rating: {rating}"
                new_label = tk.Label(self.all_tabs[index], text=label_text)
                new_label.pack()
                self.average_rating_labels.append(new_label)
        
    def load_info(self):
        # Loading room information
        floor_plan_files = ("UClub/uclub_apartments.csv",
                            "20 Hawley/20hawley_apartments.csv",
                            "Twin River Commons/twin_river_commons_apartments.csv",
                           "Printing House/printing_house_apartments.csv")
        self.floor_plans = []
        for file_name in floor_plan_files:
            # We use pandas instead of csv.reader to improve memory usage and
            # handle unexpected characters
            fp_dataframe = pd.read_csv(file_name)
            self.floor_plans.append(fp_dataframe)
    
        # Loading review information
        review_files = ("Reviews/google_reviews_uclub.csv",
                        "Reviews/google_reviews_20hawley.csv",
                        "Reviews/google_reviews_twinriver.csv",
                        "Reviews/google_reviews_printinghouse.csv")
        self.reviews = []
        for file_name in review_files:
            review_dataframe = pd.read_csv(file_name)
            self.reviews.append(review_dataframe)
        

    def import_info(self):
        # Adding the information we loaded using load_info() into tables, to be packed later
        # Use the syntax mytable.insert("", tk.END, values=(column1value, column2value, ...))
        
        # We start with the floor plans.
        self.fp_tables = []
        
        for index, floor_plan in enumerate(self.floor_plans):
            # We choose the tab we want to add the table to by the floor plan's list index
            current_tab = self.all_tabs[index]
            table_headings = floor_plan.columns.tolist()
            new_table = ttk.Treeview(current_tab, columns=table_headings, show="headings")
            for heading in table_headings:
                new_table.heading(heading, text=heading)
            for index, row in floor_plan.iterrows():
                row = row.tolist()
                new_table.insert("", tk.END, values=row)
            new_table.pack()
            self.fp_tables.append(new_table)
        
        # We move onto reviews.
        # We should plan on adding the reviews to a canvas, to make them scrollable.
        self.review_tables = []
        
        for index, review_data in enumerate(self.reviews):
            current_tab = self.all_tabs[index]
            table_headings = review_data.columns.tolist()
            new_table = ttk.Treeview(current_tab, columns=table_headings, show="headings")
            for heading in table_headings:
                new_table.heading(heading, text=heading)
            for index, row in review_data.iterrows():
                row = row.tolist()
                new_table.insert("", tk.END, values=row)
            new_table.pack()
            self.review_tables.append(new_table)
        
#     This used to have a function, but now we use it as an example for how to create and insert
#     data into tables.
#     def init_tables(self):
#         for tab in self.all_tabs:
#             new_table = ttk.Treeview(tab, columns=columns, show="headings")
#             for column in columns:
#                 new_table.heading(column, text=column)
#             new_table.insert("", tk.END, values=("value1", "value2", ...))
#             new_table.pack()
#             self.all_tables.append(new_table)

    def sort_column(self):
        # Intended to add a "click column to sort" functionality to tables
        pass
    
    def get_random_review(self):
        # Intended to be bound to a button to get a random review for a location
        pass
    
    def mainloop(self):
        self.root.mainloop()

window = MainWindow()
window.mainloop()